## SQL Workflow based on own understanding

1. Requisite for SQL engine
1. LocalSession binded to engine
1. Metadata structure binded to engine
1. API call that depends on LocalSession

## Database Connection with ORM SQL Alchemy

In [ ]:
# database.py

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

# Create SQL Alchemy Engine
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread":False}
)

# Create session local class with the binding to engine
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create base for future inheritance
Base = declarative_base()

## Create class for tables

- defined table schemas in models.py

In [ ]:
# models.py

from sqlalchemy import Boolean, Column, Integer, String
from database import Base

class Toodos(Base):
    __tablename__ = "todos"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    description = Column(String)
    priority = Column(Integer)
    complete = Column(Boolean, default=False)

## Create database from the main py file

- cd to the main.py folder
- uvicorn main:app --reload

In [ ]:
# main.py 
from fastapi import FastAPI
import models
from database import engine

app = FastAPI()

# Bind the metadata with the engine
models.Base.metadata.create_all(bind=engine)

@app.get("/")
async def create_database():
    return {"Database":"Created"}

## Install SQLlite 3 with CLI

- https://www.sqlite.org/download.html
- sqlite-tools-win32-x86-3380500.zip
- A bundle of command-line tools for managing SQLite database files, including the command-line shell program, the sqldiff.exe program, and the sqlite3_analyzer.exe program.
- Copy the extracted files into C:/ and rename folder as sqlite3
- Add into environmental PATH
- sqlite3 should work on the Cli

### Basic Operations 101

#### INSERT

    INSERT INTO todos(title, description, priority, complete)
    VALUES ("Go to Store", "Pick up Eggs", 4, False);

#### SELECT

    SELECT * FROM todos
    WHERE title = 'Feed dog';

#### UPDATE

    UPDATE todos SET complete=True
    WHERE id=5;

#### DELETE

    DELETE FROM todos 
    WHERE id=5;

#### Useful Commands
    .schema
    .mode column # formatted results
    .mode markdown # formatted to markdown
    .mode box # Within box
    .mode table # Tabulated format

## Implementing the query into API to read all records

In [ ]:
from fastapi import FastAPI, Depends
import models
from database import engine, SessionLocal
from sqlalchemy.orm import Session

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

# try to get database with local session, and close 
def get_db():
    try:
        db=SessionLocal()
        yield db
    finally:
        db.close

# Session depends on get_db function and run query against it to return all
@app.get("/")
async def read_all(db:Session=Depends(get_db)):
    return db.query(models.Todos).all()

## API to retrive 1 record (GET)

In [ ]:
from fastapi import FastAPI, Depends, HTTPException

@app.get("/todo/{todo_id}")
async def read_todo(todo_id:int, db:Session = Depends(get_db)):
    todo_model = db.query(models.Todos)\
        .filter(models.Todos.id == todo_id)\
        .first()
    if todo_model is not None:
        return todo_model
    raise http_exception

def http_exception():
    return HTTPException(status_code=404, detail="Item not Found")

## API to Add Record (POST)

In [ ]:
# main.py

from typing import Optional
from pydantic import BaseModel, Field

class ToDo(BaseModel):
    title: str
    description: Optional[str]
    priority: int = Field(gt=0, lt=6, description="Priority btw 1-5")
    complete: bool

@app.post("/")
async def create_todo(todo: ToDo, db: Session = Depends(get_db)):
    todo_model = models.Todos()
    todo_model.titel = todo.title
    todo_model.description = todo.description
    todo_model.priority = todo.priority
    todo_model.complete = todo.complete

    db.add(todo_model)
    db.commit()

    return {
        "status": 201,
        "transaction": "successful"
    }

## API to Update Record (PUT)

In [ ]:
@app.put("/{todo_id}")
async def update_todo(todo_id: int, todo: ToDo, db: Session = Depends(get_db)):
    todo_model = db.query(models.Todos).filter(models.Todos.id == todo_id).first()

    if todo_model is None:
        raise http_exception()
    todo_model.title = todo.title
    todo_model.description = todo.description
    todo_model.priority = todo.priority
    todo_model.complete = todo.complete

    db.add(todo_model)
    db.commit()

    return {
        "status": 200,
        "transaction": "successful"
    }

## API to delete record (DELETE)

In [ ]:
@app.delete("/{todo_id}")
async def delete_todo(todo_id: int, db: Session = Depends(get_db)):
    todo_model = db.query(models.Todos).filter(models.Todos.id == todo_id).first()

    if todo_model is None:
        raise http_exception()
    
    db.query(models.Todos).filter(models.Todos.id == todo_id).delete()

    db.commit()

    return successful_response(200)

# updates made for reusability of success code
def successful_response(status_code:int):
    return {
        "status": status_code,
        "transaction": "successful"
    }